In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# 1) Synthetic stock price data
np.random.seed(0)
dates = pd.date_range(start='2021-01-01', periods=200, freq='B')
trend = np.linspace(50, 150, 200)
seasonal = 10 * np.sin(np.arange(200) * 2 * np.pi / 20)
noise = np.random.normal(scale=2, size=200)
price = trend + seasonal + noise
df = pd.DataFrame({'date': dates, 'price': price}).set_index('date')

# 2) Feature engineering
for lag in [1,2,3]:
    df[f'lag{lag}'] = df['price'].shift(lag)
df['roll_mean5'] = df['price'].rolling(5).mean()
df.dropna(inplace=True)

# 3) Train/test split
n_train = int(len(df)*0.7)
train, test = df.iloc[:n_train], df.iloc[n_train:]
X_train, y_train = train.drop('price', axis=1), train['price']
X_test, y_test = test.drop('price', axis=1), test['price']

# 4) Candidate models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=50, random_state=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, random_state=0),
    'HistGB': HistGradientBoostingRegressor(max_iter=20, learning_rate=0.1, random_state=0)
}

# 5) TimeSeriesSplit CV
tscv = TimeSeriesSplit(n_splits=3)
cv_results = {}
for name, model in models.items():
    rmses = []
    for train_idx, val_idx in tscv.split(X_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model.fit(X_tr, y_tr)
        preds = model.predict(X_val)
        rmses.append(np.sqrt(mean_squared_error(y_val, preds)))
    cv_results[name] = np.mean(rmses)

# 6) Select best
best_name = min(cv_results, key=cv_results.get)
best_model = models[best_name]
best_model.fit(X_train, y_train)
y_pred_test = best_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

# Results
results = pd.DataFrame.from_dict(cv_results, orient='index', columns=['CV_RMSE'])
results['Test_RMSE'] = np.nan
results.loc[best_name, 'Test_RMSE'] = test_rmse
results

,CV_RMSE,Test_RMSE
LinearRegression,2.682004,2.746601
RandomForest,9.323978,NaN
GradientBoosting,9.133009,NaN
HistGB,17.179023,NaN


## Lag(시차)와 Rolling(이동 통계)을 사용하는 이유

---

### 1. 왜 최근 값을 피처로 쓰나?  
- **시계열의 자기의존성(Autocorrelation)**:  
  과거 시점의 값이 미래 값에 영향을 미치는 경향이 있습니다.  
  → 최신 데이터를 이용해 예측 정확도를 높일 수 있어요.

---

### 2. Lag 특성 (lag1, lag2, lag3)  
- **lag1**: 바로 전일(1일 전) 종가  
- **lag2**: 2일 전 종가  
- **lag3**: 3일 전 종가  
- **의미**: “어제, 그저께, 그끄저께” 가격이 오늘(또는 내일) 가격에 어떤 영향을 주는지 모델이 학습

---

### 3. Rolling 통계 (roll_mean5)  
- **roll_mean5**: 과거 **5영업일**(또는 5시점) 종가의 **산술평균**  
  \[
    \text{roll\_mean5}_t = \frac{1}{5}\sum_{i=1}^{5} y_{t-i}
  \]
- **의미**:  
  - 단일 과거 값이 아니라, 최근 5일 가격의 “평균 흐름” 정보를 줌  
  - 노이즈(급등·급락) 완화 → 안정적인 추세 학습

---

### 4. 숫자(3, 5)를 정하는 기준  
- **lag의 최대 날짜(3일)**  
  - 너무 많으면 피처 차원이 늘어나고, 정보가 오래될수록 예측력 감소  
  - 실습 예제에서는 간단히 “최근 3일”로 제한  

- **rolling 윈도우 크기(5일)**  
  - 일반적으로 **주간 이동평균(5일)**이 자주 쓰임  
  - 단기 추세(daily noise 제거) 포착용

> 실제 데이터·문제에 따라  
> - lag를 1~10일,  
> - rolling을 7일·14일·30일 등으로 바꿔 가며  
> **교차검증**으로 최적 조합을 찾아야 합니다.  


